In [1]:
import pandas as pd
import requests
import numpy as np
from pandas import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
import geocoder

In [2]:
#Creating a map of Riyadh City (Saudi Arabia)
latitude = 24.74
longitude = 46.72

map_riyadh = folium.Map(location=[latitude, longitude], zoom_start=11)

#labeling the map with the district names
riyadh_dist = [
    ["New Riyadh", 24.83, 46.637],
    ["Riyadh CBD", 24.682, 46.675],
    ["Eastern Riyadh", 24.775, 46.80],
    ["Old Riyadh", 24.635, 46.837],
]
dist_radius = 115
i = 0
for row in riyadh_dist:
    folium.CircleMarker(
        [riyadh_dist[i][1], riyadh_dist[i][2]],
        radius=dist_radius,
        #popup=riyadh_dist[i][0],
        color='grey',
        fill=True,
        fill_color='black',
        fill_opacity=0.2,
        parse_html=False).add_to(map_riyadh)
    folium.Marker(location= [riyadh_dist[i][1]-0.015, riyadh_dist[i][2]],
                  icon=folium.DivIcon(
                      icon_size=(100, 100),
                      html='<div style="font-size: 14pt; font-family: Arial; color: White; text-align: center";>%s</div>' %riyadh_dist[i][0])
                 ).add_to(map_riyadh)
    i = i + 1

# display map
map_riyadh

In [3]:
## Define Foursquare Credentials and Version
CLIENT_ID = 'SADOJ4UVNRZBGKOYSRUODL5VV1F0HC3GFTRMQGKIX13O2QRW' # Foursquare ID
CLIENT_SECRET = 'SIGQP2R54TMXNSPH4FWAE1AA0VCNDBI55SAT5TBTG1IFHLZF' # Foursquare Secret
ACCESS_TOKEN = 'R4M3CXYPAKIDHNHS2OBACDLE4XUH5FQDABZLLHZI330ARGSH' #Access token
VERSION = '20210726' # Foursquare API version
LIMIT = 10000

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#Query
search_query = 'Food Truck'
radius = 20000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)

Your credentails:
CLIENT_ID: SADOJ4UVNRZBGKOYSRUODL5VV1F0HC3GFTRMQGKIX13O2QRW
CLIENT_SECRET:SIGQP2R54TMXNSPH4FWAE1AA0VCNDBI55SAT5TBTG1IFHLZF


In [4]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
dataframe = df.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe['categories'] = dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe.columns = [column.split('.')[-1] for column in dataframe.columns]

In [5]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

#adding districts of Riyadh
i = 0
for row in riyadh_dist:
    folium.CircleMarker(
        [riyadh_dist[i][1], riyadh_dist[i][2]],
        radius=dist_radius,
        #popup=riyadh_dist[i][0],
        color='grey',
        fill=True,
        fill_color='black',
        fill_opacity=0.2,
        parse_html=False).add_to(venues_map)
    folium.Marker(location= [riyadh_dist[i][1]-0.015, riyadh_dist[i][2]],
                  icon=folium.DivIcon(
                      icon_size=(100, 100),
                      html='<div style="font-size: 14pt; font-family: Arial; color: White; text-align: center";>%s</div>' %riyadh_dist[i][0])
                 ).add_to(venues_map)
    i = i + 1

# add the Food trucks as blue circle markers
for lat, lng, label in zip(dataframe.lat, dataframe.lng, dataframe.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [6]:
#Clustring the venues
k=3
riyadh_clustering = dataframe[['lat', 'lng']]
kmeans = KMeans(n_clusters = k,random_state=0).fit(riyadh_clustering)
kmeans.labels_
dataframe.insert(0, 'Cluster Labels', kmeans.labels_)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


#adding districts of Riyadh
i = 0
for row in riyadh_dist:
    folium.CircleMarker(
        [riyadh_dist[i][1], riyadh_dist[i][2]],
        radius=dist_radius,
        #popup=riyadh_dist[i][0],
        color='grey',
        fill=True,
        fill_color='black',
        fill_opacity=0.2,
        parse_html=False).add_to(map_clusters)
    folium.Marker(location= [riyadh_dist[i][1]-0.015, riyadh_dist[i][2]],
                  icon=folium.DivIcon(
                      icon_size=(100, 100),
                      html='<div style="font-size: 14pt; font-family: Arial; color: White; text-align: center";>%s</div>' %riyadh_dist[i][0])
                 ).add_to(map_clusters)
    i = i + 1

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(dataframe['lat'], dataframe['lng'], dataframe['name'], dataframe['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# display map
map_clusters

In [8]:
#retriving the data from Th Municipality - for Licensed venues 


url = 'https://raw.githubusercontent.com/jalharoon/Coursera_Capstone/main/project/Municipality_data.csv'
lic_df = pd.read_csv(url)
lic_df.drop(lic_df[lic_df['Licensed'] <= 0].index, inplace = True)

lic_filtered_df = lic_df[['Licensed','Latitude','Longitude']]
licensed_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

#adding districts of Riyadh
i = 0
for row in riyadh_dist:
    folium.CircleMarker(
        [riyadh_dist[i][1], riyadh_dist[i][2]],
        radius=dist_radius,
        color='grey',
        fill=True,
        fill_color='black',
        fill_opacity=0.2,
        parse_html=False).add_to(licensed_map)
    folium.Marker(location= [riyadh_dist[i][1]-0.015, riyadh_dist[i][2]],
                  icon=folium.DivIcon(
                      icon_size=(100, 100),
                      html='<div style="font-size: 14pt; font-family: Arial; color: White; text-align: center";>%s</div>' %riyadh_dist[i][0])
                 ).add_to(licensed_map)
    i = i + 1

#markers
for lat, lng, label in zip(lic_filtered_df.Latitude, lic_filtered_df.Longitude, lic_filtered_df.Licensed):
    if label > 30:
        folium.CircleMarker(
            [lat, lng],
            radius= int(label*0.6),
            color='lime',
            popup=str(label),
            fill = True,
            fill_color='lime',
            fill_opacity=0.4
        ).add_to(licensed_map)
    else:
        folium.CircleMarker(
            [lat, lng],
            radius= int(label*2),
            color='green',
            popup=str(label),
            fill = True,
            fill_color='green',
            fill_opacity=0.4
        ).add_to(licensed_map) 

#dsplaying the map
licensed_map

In [7]:
#retriving the data from Th Municipality - for availabe venues 
url = 'https://raw.githubusercontent.com/jalharoon/Coursera_Capstone/main/project/Municipality_data.csv'
avi_df = pd.read_csv(url)
avi_df.drop(avi_df[avi_df['Available'] <= 0].index, inplace = True)

avi_filtered_df = avi_df[['Available','Latitude','Longitude']]
avilable_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

#adding districts of Riyadh
i = 0
for row in riyadh_dist:
    folium.CircleMarker(
        [riyadh_dist[i][1], riyadh_dist[i][2]],
        radius=dist_radius,
        color='grey',
        fill=True,
        fill_color='black',
        fill_opacity=0.2,
        parse_html=False).add_to(avilable_map)
    folium.Marker(location= [riyadh_dist[i][1]-0.015, riyadh_dist[i][2]],
                  icon=folium.DivIcon(
                      icon_size=(100, 100),
                      html='<div style="font-size: 14pt; font-family: Arial; color: White; text-align: center";>%s</div>' %riyadh_dist[i][0])
                 ).add_to(avilable_map)
    i = i + 1

#markers
for lat, lng, label in zip(avi_filtered_df.Latitude, avi_filtered_df.Longitude, avi_filtered_df.Available):
    if label > 30:
        folium.CircleMarker(
            [lat, lng],
            radius= int(label*0.6),
            color='red',
            popup=str(label),
            fill = True,
            fill_color='red',
            fill_opacity=0.4
        ).add_to(avilable_map)
    else:
        folium.CircleMarker(
            [lat, lng],
            radius= int(label*2),
            color='blue',
            popup=str(label),
            fill = True,
            fill_color='blue',
            fill_opacity=0.4
        ).add_to(avilable_map) 
    
#dsplaying the map
avilable_map

In [9]:
#data for the report
avi_df.head()

,Municipality,Dist,Address,Cap,Licensed,Available,Full Address,Latitude,Longitude
0,الشفاء,عكاظ,منتزه عكاض,5,2,3,منتزه عكاض حي عكاظ الرياض,24.525134,46.670098
1,الشفاء,بدر,طريق بن تيمية,4,2,2,طريق بن تيمية حي بدر الرياض,24.542688,46.692135
2,الشفاء,بدر,حديقة بدر,3,0,3,حديقة بدر حي بدر الرياض,24.550120,46.700257
3,الشفاء,الشفا,طريق بن تيميه,5,1,4,طريق بن تيميه حي الشفا الرياض,24.555617,46.701945
4,العزيزية,الدار البيضاء,ممشى حي الدار البيضاء,2,0,2,ممشى حي الدار البيضاء حي الدار البيضاء الرياض,24.562333,46.776440


In [10]:
#data for the report
avi_df.shape

(52, 9)

In [11]:
#data for the report
dataframe

,Cluster Labels,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,crossStreet,neighborhood,id
0,1,Food Truck Gathering at Riyadh Public Transpor...,Street Food Gathering,24.724280,46.699453,"[{'label': 'display', 'lat': 24.72428001324195...",2716,12432,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[الرياض 12432, المملكة العربية السعودية]",NaN,NaN,NaN,5a54d94bf193c011e0f55921
1,1,Blïnd Burger Food Truck,Food Truck,24.734508,46.698943,"[{'label': 'display', 'lat': 24.734508, 'lng':...",2214,12435,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[الرياض 12435, المملكة العربية السعودية]",NaN,NaN,NaN,5bf6f348ee7120002c9e6471
2,2,Urth Caffé - Food Truck,Café,24.750099,46.725397,"[{'label': 'display', 'lat': 24.750099, 'lng':...",1249,12451,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,[Riyadh International Convention & Exhibition ...,Riyadh International Convention & Exhibition C...,NaN,NaN,6045e4f1071ccf469de370bb
3,0,Food Trucks @ Turki Alawwal (عربات طعام تركي ا...,Food Truck,24.785707,46.615016,"[{'label': 'display', 'lat': 24.78570668886652...",11768,NaN,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[Anas bin Malek, الرياض, المملكة العربية السعو...",NaN,Anas bin Malek,NaN,579bbb6d498e5aa176ef3670
4,0,Brunch (Food Truck),Food Truck,24.783903,46.592046,"[{'label': 'display', 'lat': 24.78390294614674...",13826,13522,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[الرياض 13522, المملكة العربية السعودية]",NaN,NaN,NaN,5d7ac122023689000785eca9
5,1,حنيني - Food Truck,Food Truck,24.731597,46.650543,"[{'label': 'display', 'lat': 24.73159691539122...",7084,12363,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[طريق التخصصي (حي المحمدية), الرياض 12363, الم...",طريق التخصصي,حي المحمدية,NaN,5f8a170633ca484145aa2925
6,0,Protein_shot Food Truck,Food Truck,24.811647,46.592809,"[{'label': 'display', 'lat': 24.81164676024775...",15128,13521,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[الرياض 13521, المملكة العربية السعودية]",NaN,NaN,NaN,5a9589ce8a6f175a5f02a1a2
7,1,Healthy Food Truck,Food Truck,24.723333,46.697860,"[{'label': 'display', 'lat': 24.72333308428604...",2907,NaN,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[Al Orouba Road, الرياض, المملكة العربية السعو...",Al Orouba Road,NaN,NaN,5aae47bbe1f22809dc546c42
8,2,Brunch (Food Truck),Food Truck,24.716922,46.737288,"[{'label': 'display', 'lat': 24.71692164757766...",3107,12321,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[الرياض 12321, المملكة العربية السعودية]",NaN,NaN,NaN,5a11d70be179100ce1c37a3f
9,0,4Twins - Food Truck,Food Truck,24.778656,46.585865,"[{'label': 'display', 'lat': 24.77865608320065...",14225,13521,SA,الرياض,منطقة الرياض‎,المملكة العربية السعودية,"[الرياض 13521, المملكة العربية السعودية]",NaN,NaN,NaN,5ef916217f05c700088c4bf0


In [12]:
#data for the report
dataframe.shape

(50, 17)